In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

from sklearn.metrics import r2_score

In [58]:
from google.colab import files 


# Define hyperparameters and other important variables

In [59]:
target_variable = "GloblRad"

timestamp_col = "date"

target_sequence_length = 1*2

test_size = 0.20

first_day_test = "2022-11-06"

last_day_test = "2022-12-30"
in1 = 24
hyperparameters = {
    "in_length" : 1*in1,
    "step_size" : 1,
    "n_estimators" : 12,
    "max_depth" : 5,
    "subsample" : 0.5,
    "min_child_weight" : 1,
    "selected_features" : [target_variable]
    }

# Prepare data

First, we load the data and split it into train and test. It is important that the shuffle argument is False, because we need to maintain the original order of the data points in the data. 

In [60]:
df = pd.read_csv("./data.csv")
df = df.set_index('date')
df = df.set_index(pd.DatetimeIndex(pd.to_datetime(df.index)))

<ipython-input-60-99c1576d12bc>:1: DtypeWarning: Columns (2,3,5,6,9,10,12,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./data.csv")


In [61]:
df

,Unnamed: 0,ObsTime,StnPres,SeaPres,Temperature,Td dew point,RH,WS,WD,WSGust,WDGust,Precp,PrecpHour,SunShine,GloblRad,Visb,UVI,Cloud Amount
date,,,,,,,,,,,,,,,,,,
2015-01-01,0,1,1022.3,1026.0,16.0,7.9,59,4.9,80,10.7,80,0.0,0.0,...,0.0,...,...,...
2015-01-01,1,2,1022.6,1026.3,15.5,7.8,60,4.6,70,10.9,90,0.0,0.0,...,0.0,5.0,...,5.0
2015-01-01,2,3,1022.8,1026.5,15.3,7.4,59,4.3,70,11.5,70,0.0,0.0,...,0.0,...,...,...
2015-01-01,3,4,1022.9,1026.6,14.8,7.2,60,4.1,70,11.0,60,0.0,0.0,...,0.0,...,...,...
2015-01-01,4,5,1023.4,1027.1,14.4,6.7,60,3.5,70,9.7,90,0.0,0.0,...,0.0,7.0,...,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-01,19,20,1002.0,1005.4,29.9,23.9,70,2.4,270,6.5,280,0.0,0.0,...,0.0,...,0,...
2022-06-01,20,21,1002.7,1006.1,29.3,24.2,74,1.9,280,6.5,290,0.0,0.0,...,0.0,...,0,...
2022-06-01,21,22,1003.2,1006.6,29.1,24.7,77,1.3,230,4.6,260,0.0,0.0,...,0.0,...,0,...


In [62]:
# shuffled_data.to_csv("5T_Training_Data_0330_1231_Charger_weekday_shuffled.csv")

In [63]:
df = df[['GloblRad','Temperature','RH','StnPres']]
# df = df[['solarenergy','temp','humidity']]
df = df.replace('X',np.nan)
df = df.dropna()
df = df.astype('float32')

In [64]:
training_data = df[df.index < first_day_test]
training_data

,GloblRad,Temperature,RH,StnPres
date,,,,
2015-01-01,0.0,16.000000,59.0,1022.299988
2015-01-01,0.0,15.500000,60.0,1022.599976
2015-01-01,0.0,15.300000,59.0,1022.799988
2015-01-01,0.0,14.800000,60.0,1022.900024
2015-01-01,0.0,14.400000,60.0,1023.400024
...,...,...,...,...
2022-06-01,0.0,29.900000,70.0,1002.000000
2022-06-01,0.0,29.299999,74.0,1002.700012
2022-06-01,0.0,29.100000,77.0,1003.200012


In [65]:
test_data = df[df.index >= first_day_test]
test_data = test_data[test_data.index <= last_day_test]
test_data

,GloblRad,Temperature,RH,StnPres
date,,,,


In [66]:
permuted_data = df
test_index = int(len(df) *0.8) 
test_index

51770

In [67]:
# Split the data into a test and a train set
training_data = permuted_data[:test_index]
test_data = permuted_data[test_index:]

In [68]:
test_data

,GloblRad,Temperature,RH,StnPres
date,,,,
2020-12-09,0.0,20.299999,100.0,1012.500000
2020-12-09,0.0,20.500000,100.0,1012.700012
2020-12-09,0.0,20.799999,99.0,1013.099976
2020-12-09,0.0,20.600000,100.0,1013.099976
2020-12-09,0.0,20.700001,100.0,1012.599976
...,...,...,...,...
2022-06-01,0.0,29.900000,70.0,1002.000000
2022-06-01,0.0,29.299999,74.0,1002.700012
2022-06-01,0.0,29.100000,77.0,1003.200012


In [69]:
import numpy as np
from typing import Tuple
import pandas as pd

def get_xgboost_x_y(
    indices: list, 
    data: np.array,
    target_sequence_length,
    input_seq_len: int
    ) -> Tuple[np.array, np.array]:
    print(indices)
    print("Preparing data..")
    # Loop over list of training indices
    for i, idx in enumerate(indices):
        # Slice data into instance of length input length + target length
        data_instance = data[idx[0]:idx[1]]
        
        x = data_instance[0:input_seq_len]
        assert len(x) == input_seq_len
        y = data_instance[input_seq_len:input_seq_len+target_sequence_length]
        y = y[:,0]
        
        # Create all_y and all_x objects in first loop iteration
        if i == 0:
            all_y = y.reshape(1, -1)
            all_x = x.reshape(1, -1)
        else:
            all_y = np.concatenate((all_y, y.reshape(1, -1)), axis=0)
            all_x = np.concatenate((all_x, x.reshape(1, -1)), axis=0)
    print("Finished preparing data!")
    return all_x, all_y


def get_indices_entire_sequence(
    data: pd.DataFrame, 
    window_size: int, 
    step_size: int
    ) -> list:
        stop_position = len(data)-1 # 1- because of 0 indexing
        # Start the first sub-sequence at index position 0
        subseq_first_idx = 0
        
        subseq_last_idx = window_size
        
        indices = []
        
        while subseq_last_idx <= stop_position:

            indices.append((subseq_first_idx, subseq_last_idx))
            # print(subseq_first_idx,subseq_last_idx)
            subseq_first_idx += step_size
            
            subseq_last_idx += step_size
            
            
        return indices


In [70]:
training_data

,GloblRad,Temperature,RH,StnPres
date,,,,
2015-01-01,0.00,16.000000,59.0,1022.299988
2015-01-01,0.00,15.500000,60.0,1022.599976
2015-01-01,0.00,15.300000,59.0,1022.799988
2015-01-01,0.00,14.800000,60.0,1022.900024
2015-01-01,0.00,14.400000,60.0,1023.400024
...,...,...,...,...
2020-12-09,0.60,19.700001,99.0,1013.500000
2020-12-09,0.60,20.000000,100.0,1012.599976
2020-12-09,0.38,20.299999,99.0,1011.900024


In [71]:
# Create indices. Must be passed to function that creates (X,Y) pairs so that
# it knows where to slice the data
training_indices = get_indices_entire_sequence(
    data=training_data, 
    window_size=hyperparameters["in_length"]+target_sequence_length, 
    step_size=hyperparameters["step_size"]
    )
# Obtain (X,Y) pairs of training data
x_train, y_train = get_xgboost_x_y(
    indices=training_indices, 
    # data=training_data[hyperparameters["selected_features"]].to_numpy(),
    data=training_data.to_numpy(),
    target_sequence_length=target_sequence_length,
    input_seq_len=hyperparameters["in_length"]
    )

test_indices = get_indices_entire_sequence(
    data=test_data, 
    window_size=hyperparameters["in_length"]+target_sequence_length, 
    step_size=288
    )
# Obtain (X,Y) pairs of test data
x_test, y_test = get_xgboost_x_y(
    indices=test_indices, 
    # data=test_data[hyperparameters["selected_features"]].to_numpy(),
    data=test_data.to_numpy(),
    target_sequence_length=target_sequence_length,
    input_seq_len=hyperparameters["in_length"]
    )

[(0, 26), (1, 27), (2, 28), (3, 29), (4, 30), (5, 31), (6, 32), (7, 33), (8, 34), (9, 35), (10, 36), (11, 37), (12, 38), (13, 39), (14, 40), (15, 41), (16, 42), (17, 43), (18, 44), (19, 45), (20, 46), (21, 47), (22, 48), (23, 49), (24, 50), (25, 51), (26, 52), (27, 53), (28, 54), (29, 55), (30, 56), (31, 57), (32, 58), (33, 59), (34, 60), (35, 61), (36, 62), (37, 63), (38, 64), (39, 65), (40, 66), (41, 67), (42, 68), (43, 69), (44, 70), (45, 71), (46, 72), (47, 73), (48, 74), (49, 75), (50, 76), (51, 77), (52, 78), (53, 79), (54, 80), (55, 81), (56, 82), (57, 83), (58, 84), (59, 85), (60, 86), (61, 87), (62, 88), (63, 89), (64, 90), (65, 91), (66, 92), (67, 93), (68, 94), (69, 95), (70, 96), (71, 97), (72, 98), (73, 99), (74, 100), (75, 101), (76, 102), (77, 103), (78, 104), (79, 105), (80, 106), (81, 107), (82, 108), (83, 109), (84, 110), (85, 111), (86, 112), (87, 113), (88, 114), (89, 115), (90, 116), (91, 117), (92, 118), (93, 119), (94, 120), (95, 121), (96, 122), (97, 123), (98, 

In [72]:
from sklearn.model_selection import train_test_split

X_train, X_valid, Y_train, Y_valid = train_test_split(x_train, y_train, test_size=0.2)

In [73]:
import tensorflow as tf

In [74]:
NB_classes = 2 # 輸出數
NB_neurones = 30 # 主要神經元數量
NB_features = in1*4 # 輸入數
activation_func = tf.keras.activations.relu # 激活函數

model = tf.keras.Sequential([
    tf.keras.layers.Dense(NB_neurones, activation=activation_func, input_shape=(NB_features,)), # 輸入層
    tf.keras.layers.Dense(NB_neurones, activation=activation_func), # 隱藏層
    tf.keras.layers.Dense(NB_neurones, activation=activation_func), 
    tf.keras.layers.Dense(NB_neurones, activation=activation_func), 
    tf.keras.layers.Dense(int(NB_neurones/2), activation=activation_func), # 改變神經元數量
    tf.keras.layers.Dense(int(NB_neurones/2), activation=activation_func), 
    tf.keras.layers.Dense(NB_neurones, activation=activation_func), 
    tf.keras.layers.Dense(NB_neurones, activation=activation_func), 
    tf.keras.layers.Dense(int(NB_neurones*2), activation=activation_func), # 改變神經元數量
    tf.keras.layers.Dense(int(NB_neurones*2), activation=activation_func), 
    tf.keras.layers.Dense(NB_neurones, activation=activation_func), 
    tf.keras.layers.Dense(NB_neurones, activation=activation_func),
    tf.keras.layers.Dropout(0.3), # 新增 Dropout 層
    tf.keras.layers.Dense(NB_neurones, activation=activation_func), 
    tf.keras.layers.Dense(NB_neurones, activation=activation_func), 
    tf.keras.layers.Dense(NB_neurones, activation=activation_func),
    tf.keras.layers.Dropout(0.4), # Dropout 層
    tf.keras.layers.Dense(NB_classes, activation=tf.keras.activations.softmax) # 輸出層
])

# 編譯模型
model.compile(optimizer="adam", loss=tf.keras.losses.mse, metrics=[tf.keras.metrics.MeanSquaredError()])


# 輸出模型摘要
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_32 (Dense)            (None, 30)                2910      
                                                                 
 dense_33 (Dense)            (None, 30)                930       
                                                                 
 dense_34 (Dense)            (None, 30)                930       
                                                                 
 dense_35 (Dense)            (None, 30)                930       
                                                                 
 dense_36 (Dense)            (None, 15)                465       
                                                                 
 dense_37 (Dense)            (None, 15)                240       
                                                                 
 dense_38 (Dense)            (None, 30)               

In [75]:
model.fit(x=X_train,
          y=Y_train,
          epochs=20,
          validation_data=(X_valid,Y_valid),
          verbose=1,
          shuffle=True) #Train our model

Epoch 1/20
1294/1294 [==============================] - 5s 2ms/step - loss: 0.7427 - mean_squared_error: 0.7427 - val_loss: 0.7243 - val_mean_squared_error: 0.7243
Epoch 2/20
1294/1294 [==============================] - 3s 3ms/step - loss: 0.7401 - mean_squared_error: 0.7401 - val_loss: 0.7243 - val_mean_squared_error: 0.7243
Epoch 3/20
1294/1294 [==============================] - 3s 2ms/step - loss: 0.7401 - mean_squared_error: 0.7401 - val_loss: 0.7243 - val_mean_squared_error: 0.7243
Epoch 4/20
1294/1294 [==============================] - 3s 2ms/step - loss: 0.7401 - mean_squared_error: 0.7401 - val_loss: 0.7243 - val_mean_squared_error: 0.7243
Epoch 5/20
1294/1294 [==============================] - 3s 2ms/step - loss: 0.7401 - mean_squared_error: 0.7401 - val_loss: 0.7243 - val_mean_squared_error: 0.7243
Epoch 6/20
1294/1294 [==============================] - 3s 3ms/step - loss: 0.7401 - mean_squared_error: 0.7401 - val_loss: 0.7243 - val_mean_squared_error: 0.7243
Epoch 7/20
1294/

In [76]:
test_forecasts = model.predict(x_test)
test_mae = mean_squared_error(y_test, test_forecasts)
test_mse = mean_absolute_error(y_test, test_forecasts)
test_r2 = r2_score(y_test, test_forecasts)
print("Mae",test_mae)
print("Mse",test_mse)
print("R2",test_r2)

print("Mean test data value: {}".format(np.mean(y_test)))

2/2 [==============================] - 0s 3ms/step
Mae 0.20265591
Mse 0.43810523
R2 -177.93659424227897
Mean test data value: 0.07733333855867386


In [77]:
converter = tf.lite.TFLiteConverter.from_keras_model(model) #create a converter
tflite_model = converter.convert() #convert the model without quantization (for one of the next video)


open("/content/tflite_model.tflite","wb").write(tflite_model) #Create a file containing our tflite model

83412

In [78]:
!apt-get install -qq xxd #installing the tool

In [79]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat /content/tflite_model.tflite | xxd -i >> /content/model.h #create an hexadecimal array containing all our parameters
!echo "};" >> /content/model.h

files.download("/content/model.h") #automaticly download your file

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>